### 1 - Test Example

In [1]:
import numpy as np
import pyomo.environ as pyEnv

In [2]:
# dados falsos
areas = [
    [[1, 2, 3, 4], [5, 3, 2, 1]],
    [[8, 5, 2, 1], [3, 4, 7, 6]]
]
matrix = np.array(areas)
a = [8, 8]  # excedente de bicicletas disponíveis
s = [1, 1]   # espaço ocupado por cada categoria de bicicleta
T = 15      # capacidade total do caminhão

model = pyEnv.ConcreteModel()

# índices para as áreas e categorias
model.I = pyEnv.Set(initialize=range(matrix.shape[0]))
model.J = pyEnv.Set(initialize=range(matrix.shape[1]))
model.K = pyEnv.Set(initialize=range(matrix.shape[2]))

# variáveis de decisão
model.x = pyEnv.Var(model.I, model.J, model.K, domain=pyEnv.Binary, initialize=0)

# função objetivo
model.profit = pyEnv.Objective(
    expr=sum(matrix[i, j, k] * model.x[i,j, k] for i in model.I for j in model.J for k in model.K),
    sense=pyEnv.maximize
)

# restrições
def supply_constraint(model, j):
    return sum(model.x[i, j, k] for i in model.I for k in model.K) <= a[j]

model.supply_constraint = pyEnv.Constraint(model.J, rule=supply_constraint)

def capacity_constraint(model):
    return sum(s[j] * model.x[i, j, k] for i in model.I for j in model.J for k in model.K) <= T

model.capacity_constraint = pyEnv.Constraint(rule=capacity_constraint)

# adicionar restrição condicional
def sequential_constraint_rule(model, i, j, k):
    if k == 0:
        return pyEnv.Constraint.Skip
    else:
        return model.x[i, j, k] <= model.x[i, j, k-1]

model.sequential_constraint = pyEnv.Constraint(model.I, model.J, model.K, rule=sequential_constraint_rule)


# resolver o modelo
solver = pyEnv.SolverFactory('glpk')
results = solver.solve(model)

# resultados
print("Status:", results.solver.termination_condition)
print("Valor objetivo ótimo (soma dos lucros esperados):", pyEnv.value(model.profit))

print("\nSolução ótima:")
for i in model.I:
    for j in model.J:
        for k in model.K:
            print(f"{i,j,k}: {model.x[i, j, k].value}")


Status: optimal
Valor objetivo ótimo (soma dos lucros esperados): 56.0

Solução ótima:
(0, 0, 0): 1.0
(0, 0, 1): 1.0
(0, 0, 2): 1.0
(0, 0, 3): 1.0
(0, 1, 0): 1.0
(0, 1, 1): 1.0
(0, 1, 2): 1.0
(0, 1, 3): 1.0
(1, 0, 0): 1.0
(1, 0, 1): 1.0
(1, 0, 2): 1.0
(1, 0, 3): 0.0
(1, 1, 0): 1.0
(1, 1, 1): 1.0
(1, 1, 2): 1.0
(1, 1, 3): 1.0


In [3]:
# quantidade de bicicletas de cada categoria j transportadas para cada área i
sum_x = [[sum(model.x[i, j, k].value for k in model.K) for j in model.J] for i in model.I]
sum_x

[[4.0, 4.0], [3.0, 4.0]]

In [4]:
matrix

array([[[1, 2, 3, 4],
        [5, 3, 2, 1]],

       [[8, 5, 2, 1],
        [3, 4, 7, 6]]])